# Retrieval Augmentation for GPT-4 using Pinecone

#### Fixing LLMs that Hallucinate

In this notebook we will learn how to query relevant contexts to our queries from Pinecone, and pass these to a GPT-4 model to generate an answer backed by real data sources.

GPT-4 is a big step up from previous OpenAI completion models. It also exclusively uses the `ChatCompletion` endpoint, so we must use it in a slightly different way to usual. However, the power of the model makes the change worthwhile, particularly when augmented with an external knowledge base like the Pinecone vector database.

Required installs for this notebook are:

In [1]:
!pip install -qU bs4 tiktoken openai langchain pinecone-client[grpc]

## Preparing the Data

In this example, we will download the LangChain docs from [langchain.readthedocs.io/](https://langchain.readthedocs.io/latest/en/). We get all `.html` files located on the site like so:

In [17]:
!wget -r -A.html -P rtdocs https://api.python.langchain.com/en/latest

--2024-05-30 21:06:15--  https://api.python.langchain.com/en/latest
Resolving api.python.langchain.com (api.python.langchain.com)... 

104.17.33.82, 104.17.32.82, 2606:4700::6811:2152, ...
Connecting to api.python.langchain.com (api.python.langchain.com)|104.17.33.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://api.python.langchain.com/en/latest/ [following]
--2024-05-30 21:06:15--  https://api.python.langchain.com/en/latest/
Reusing existing connection to api.python.langchain.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘rtdocs/api.python.langchain.com/en/latest.tmp’

api.python.langchai     [ <=>                ]     900  --.-KB/s    in 0s      

2024-05-30 21:06:16 (20.0 MB/s) - ‘rtdocs/api.python.langchain.com/en/latest.tmp’ saved [900]

Loading robots.txt; please ignore errors.
--2024-05-30 21:06:16--  https://api.python.langchain.com/robots.txt
Reusing existing connection to api.python.langchain.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘rtdocs/api.python.langc

This downloads all HTML into the `rtdocs` directory. Now we can use LangChain itself to process these docs. We do this using the `ReadTheDocsLoader` like so:

In [2]:
pip install -U langchain-community

  Using cached langchain_community-0.2.1-py3-none-any.whl.metadata (8.9 kB)
Using cached langchain_community-0.2.1-py3-none-any.whl (2.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.document_loaders import ReadTheDocsLoader

loader = ReadTheDocsLoader('rtdocs')
docs = loader.load()
len(docs)

4462

This leaves us with hundreds of processed doc pages. Let's take a look at the format each one contains:

In [4]:
docs[0]

Document(page_content='langchain_robocorp 0.0.9¶\nlangchain_robocorp.toolkits¶\nRobocorp Action Server toolkit.\nClasses¶\ntoolkits.ActionServerRequestTool\nRequests POST tool with LLM-instructed extraction of truncated responses.\ntoolkits.ActionServerToolkit\nToolkit exposing Robocorp Action Server provided actions as individual tools.\ntoolkits.RunDetailsCallbackHandler(run_details)\nCallback handler to add run details to the run.\ntoolkits.ToolArgs\nTool arguments.\ntoolkits.ToolInputSchema\nTool input schema.', metadata={'source': 'rtdocs/api.python.langchain.com/en/latest/robocorp_api_reference.html'})

We access the plaintext page content like so:

In [5]:
print(docs[0].page_content)

langchain_robocorp 0.0.9¶
langchain_robocorp.toolkits¶
Robocorp Action Server toolkit.
Classes¶
toolkits.ActionServerRequestTool
Requests POST tool with LLM-instructed extraction of truncated responses.
toolkits.ActionServerToolkit
Toolkit exposing Robocorp Action Server provided actions as individual tools.
toolkits.RunDetailsCallbackHandler(run_details)
Callback handler to add run details to the run.
toolkits.ToolArgs
Tool arguments.
toolkits.ToolInputSchema
Tool input schema.


In [6]:
print(docs[5].page_content)

langchain_experimental 0.0.59¶
langchain_experimental.agents¶
Agent is a class that uses an LLM to choose
a sequence of actions to take.
In Chains, a sequence of actions is hardcoded. In Agents,
a language model is used as a reasoning engine to determine which actions
to take and in which order.
Agents select and use Tools and Toolkits for actions.
Functions¶
agents.agent_toolkits.csv.base.create_csv_agent(...)
Create pandas dataframe agent by loading csv to a dataframe.
agents.agent_toolkits.pandas.base.create_pandas_dataframe_agent(llm, df)
Construct a Pandas agent from an LLM and dataframe(s).
agents.agent_toolkits.python.base.create_python_agent(...)
Construct a python agent from an LLM and tool.
agents.agent_toolkits.spark.base.create_spark_dataframe_agent(llm, df)
Construct a Spark agent from an LLM and dataframe.
agents.agent_toolkits.xorbits.base.create_xorbits_agent(...)
Construct a xorbits agent from an LLM and dataframe.
langchain_experimental.autonomous_agents¶
Autonomous a

We can also find the source of each document:

In [7]:
docs[5].metadata['source'].replace('rtdocs/', 'https://')

'https://api.python.langchain.com/en/latest/experimental_api_reference.html'

We can use these to create our `data` list:

In [8]:
data = []

for doc in docs:
    data.append({
        'url': doc.metadata['source'].replace('rtdocs/', 'https://'),
        'text': doc.page_content
    })

In [60]:
data[3]

{'url': 'https://langchain.readthedocs.io/en/latest/modules/memory/types/entity_summary_memory.html',
 'text': '.ipynb .pdf Entity Memory Contents Using in a chain Inspecting the memory store Entity Memory# This notebook shows how to work with a memory module that remembers things about specific entities. It extracts information on entities (using LLMs) and builds up its knowledge about that entity over time (also using LLMs). Let’s first walk through using this functionality. from langchain.llms import OpenAI from langchain.memory import ConversationEntityMemory llm = OpenAI(temperature=0) memory = ConversationEntityMemory(llm=llm) _input = {"input": "Deven & Sam are working on a hackathon project"} memory.load_memory_variables(_input) memory.save_context( _input, {"ouput": " That sounds like a great project! What kind of project are they working on?"} ) memory.load_memory_variables({"input": \'who is Sam\'}) {\'history\': \'Human: Deven & Sam are working on a hackathon project\\nAI: 

It's pretty ugly but it's good enough for now. Let's see how we can process all of these. We will chunk everything into ~400 token chunks, we can do this easily with `langchain` and `tiktoken`:

In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

Process the `data` into more chunks using this approach.

In [11]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record['url']
    } for i in range(len(texts))])

/home/superjoeyd123/chatgpt-retrieval-plugin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  1%|          | 53/4462 [00:00<00:27, 159.05it/s]

100%|██████████| 4462/4462 [00:39<00:00, 113.69it/s]


Our chunks are ready so now we move onto embedding and indexing everything.

## Initialize Embedding Model

We use `text-embedding-3-small` as the embedding model. We can embed text like so:

In [66]:
import openai

# initialize openai API key
openai.api_key = "sk-..."

embed_model = "text-embedding-3-small"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In the response `res` we will find a JSON-like object containing our new embeddings within the `'data'` field.

In [67]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Inside `'data'` we will find two records, one for each of the two sentences we just embedded. Each vector embedding contains `1536` dimensions (the output dimensionality of the `text-embedding-3-small` model.

In [68]:
len(res['data'])

2

In [69]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

We will apply this same embedding logic to the langchain docs dataset we've just scraped. But before doing so we must create a place to store the embeddings.

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [70]:
import pinecone

index_name = 'gpt-4-langchain-docs'

# initialize connection to pinecone
pinecone.init(
    api_key="PINECONE_API_KEY",  # app.pinecone.io (console)
    environment="PINECONE_ENVIRONMENT"  # next to API key in console
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it with OpenAI `text-embedding-3-small` built embeddings like so:

In [71]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/12 [00:00<?, ?it/s]

Now we've added all of our langchain docs to the index. With that we can move on to retrieval and then answer generation using GPT-4.

## Retrieval

To search through our documents we first need to create a query vector `xq`. Using `xq` we will retrieve the most relevant chunks from the LangChain docs, like so:

In [83]:
query = "how do I use the LLMChain in LangChain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [84]:
res

{'matches': [{'id': '1fec660b-9937-4f7e-9692-280c8cc7ce0d',
              'metadata': {'chunk': 0.0,
                           'text': '.rst .pdf Chains Chains# Using an LLM in '
                                   'isolation is fine for some simple '
                                   'applications, but many more complex ones '
                                   'require chaining LLMs - either with each '
                                   'other or with other experts. LangChain '
                                   'provides a standard interface for Chains, '
                                   'as well as some common implementations of '
                                   'chains for ease of use. The following '
                                   'sections of documentation are provided: '
                                   'Getting Started: A getting started guide '
                                   'for chains, to get you up and running '
                                   'quickly.

With retrieval complete, we move on to feeding these into GPT-4 to produce answers.

## Retrieval Augmented Generation

GPT-4 is currently accessed via the `ChatCompletions` endpoint of OpenAI. To add the information we retrieved into the model, we need to pass it into our user prompts *alongside* our original query. We can do that like so:

In [85]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [87]:
print(augmented_query)

.rst .pdf Chains Chains# Using an LLM in isolation is fine for some simple applications, but many more complex ones require chaining LLMs - either with each other or with other experts. LangChain provides a standard interface for Chains, as well as some common implementations of chains for ease of use. The following sections of documentation are provided: Getting Started: A getting started guide for chains, to get you up and running quickly. Key Concepts: A conceptual guide going over the various concepts related to chains. How-To Guides: A collection of how-to guides. These highlight how to use various types of chains. Reference: API reference documentation for all Chain classes. previous Vector DB Text Generation next Getting Started By Harrison Chase © Copyright 2022, Harrison Chase. Last updated on Mar 15, 2023.

---

.rst .pdf LLMs LLMs# Large Language Models (LLMs) are a core component of LangChain. LangChain is not a provider of LLMs, but rather provides a standard interface thr

Now we ask the question:

In [88]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

To display this response nicely, we will display it in markdown.

In [89]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

To use the LLMChain in LangChain, follow these steps:

1. Import the necessary classes:
```python
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
```

2. Create an instance of the LLM and set the configuration options:
```python
llm = OpenAI(temperature=0.9)
```

3. Create a PromptTemplate instance with the input variables and the template:
```python
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good product for {product}?",
)
```

4. Create an LLMChain instance by passing the LLM and PromptTemplate instances:
```python
llm_chain = LLMChain(llm=llm, prompt_template=prompt)
```

5. Run the LLMChain with user input:
```python
response = llm_chain.run({"product": "software development"})
```

6. Access the generated response:
```python
generated_text = response["generated_text"]
```

In this example, the LLMChain is used to generate a response by passing through the user input and formatting it using the prompt template. The response is then obtained from the LLM instance (in this case, OpenAI), and the generated text can be accessed from the response dictionary.

Let's compare this to a non-augmented query...

In [90]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

I don't know.

If we drop the `"I don't know"` part of the `primer`?

In [91]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

LangChain hasn't provided any public documentation on LLMChain, nor is there a known technology called LLMChain in their library. To better assist you, please provide more information or context about LLMChain and LangChain.

Meanwhile, if you are referring to LangChain, a blockchain-based decentralized AI language model, you can start by visiting their official website (if they have one), exploring their available resources, such as documentation and tutorials, and following any instructions on setting up their technology.

If you are looking for help with a specific language chain or model in natural language processing, consider rephrasing your question to provide more accurate information or visit relevant resources like GPT-3 or other NLP-related documentation.